In [1]:
# from __future__ import print_function

import sys

import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import pyspark.sql.functions as F

import pytz
# print(__name__)builtins
if __name__ == "builtins":
    spark = SparkSession \
        .builder \
        .appName("Python Demo") \
        .config("hive.metastore.client.factory.class",
                "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .config("spark.driver.maxResultSize", "4g") \
        .enableHiveSupport() \
        .getOrCreate()
    spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict");
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "20000000")
    databases = spark.sql("show databases")
    databases = databases.collect()
    tablelist=["`banda-etl-s3`.`t_channel_details`","`banda-etl-s3`.`t_customer_install_info`","`banda-etl-s3`.`t_customer`"];
#         select * from `banda-etl-s3`.`t_loan_app` a where a.id not in(
#                            select rel.loan_id
#                                         from `banda-etl-s3`.t_loan_tag_rel rel 
#                                         where rel.tag_id in (select id from `banda-etl-s3`.t_tag where type='SELLING_CASE' ) and  rel.status='UNACTIVE'

#                         ) and  a.`status` in ('CURRENT','PAID_OFF','OVERDUE','GRACE_PERIOD') and a.product_name='RUPIAHONE' and a.`create_time` + interval '7' hour >= date '2019-08-26' 
#                         and a.customer_id not in  (select customer_id from `banda-etl-s3`.t_test_customer)
    df1=spark.sql("""
        with t_loan_app_not_in as(
            select a.* from `banda-etl-s3`.`t_loan_app` a 
                left join  
                   (select rel.id, rel.status,rel.loan_id,tag.id as tagid from `banda-etl-s3`.t_loan_tag_rel rel 
                left join `banda-etl-s3`.t_tag tag
                   on rel.tag_id=tag.id 
                   and tag.type='SELLING_CASE'
                   where tag.id is not null ) as tmp 
                   on a.id =tmp.loan_id
                   and  tmp.status='UNACTIVE'
                where tmp.loan_id is null 
                and  a.`status` in ('CURRENT','PAID_OFF','OVERDUE','GRACE_PERIOD') and a.product_name='RUPIAHONE' and a.`create_time` + interval '7' hour >= date '2019-08-26' 
                and a.customer_id not in (select customer_id from `banda-etl-s3`.t_test_customer)
                order by a.id desc 
                    )
        select date(now()+interval '7' hour-interval '1' day) as date,
                sum(if(a.loan_type !='ROLLOVER',a.amount,0)) as total_issue_amount,

                sum(coalesce(b.principal_paid,0)) as total_paid_off_principle,
                sum(b.principal_paid) as kkk,

                sum(if(a.loan_type !='ROLLOVER',a.amount,0))-sum(coalesce(b.principal_paid,0)) as loan_balance,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(coalesce(b.principal_accr,0)) -(coalesce(b.principal_paid,0) ),0)) as total_current_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,

                (coalesce(b.principal_accr,0)) -(coalesce(b.principal_paid,0)),0)) as overdue_3_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_10_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_30_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_60_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_90_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_more_than_90_principle,

                sum(coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + b.rollover_fee_paid + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)-b.erase_amount-b.coupon_amount) as total_paid_off_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD'),(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as total_outstanding_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as total_current_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_3_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_10_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_30_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_60_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_90_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_more_than_90_amount,'total' as mark

                from `banda-etl-s3`.`t_loan_app` a
                left  join `banda-etl-s3`.`t_lpay` b
                on a.`id` = b.`loan_app_id`
                where a.`status` in ('CURRENT','PAID_OFF','OVERDUE','GRACE_PERIOD') and a.product_name='RUPIAHONE' and a.`create_time` + interval '7' hour >= date '2019-08-26'
                and a.customer_id not in (select customer_id from `banda-etl-s3`.t_test_customer)

                union all

                select date(now()+interval '7' hour-interval '1' day) as date,
                sum(if(a.loan_type !='ROLLOVER',a.amount,0)) as total_issue_amount,

                sum(coalesce(b.principal_paid,0)) as total_paid_off_principle,
                sum(b.principal_paid) as kkk,

                sum(if(a.loan_type !='ROLLOVER',a.amount,0))-sum(coalesce(b.principal_paid,0)) as loan_balance,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(coalesce(b.principal_accr,0)) -(coalesce(b.principal_paid,0) ),0)) as total_current_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,

                (coalesce(b.principal_accr,0)) -(coalesce(b.principal_paid,0)),0)) as overdue_3_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_10_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_30_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_60_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_90_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_more_than_90_principle,

                sum(coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + b.rollover_fee_paid + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)-b.erase_amount-b.coupon_amount) as total_paid_off_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD'),(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as total_outstanding_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as total_current_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_3_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_10_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_30_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_60_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_90_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_more_than_90_amount,'total_new' as mark

                from  t_loan_app_not_in  a
                left  join `banda-etl-s3`.`t_lpay` b
                on a.`id` = b.`loan_app_id`

                union all

                select date(now()+interval '7' hour-interval '1' day) as date,
                sum(if(a.loan_type !='ROLLOVER',a.amount,0)) as total_issue_amount,

                sum(coalesce(b.principal_paid,0)) as total_paid_off_principle,
                sum(b.principal_paid) as kkk,

                sum(if(a.loan_type !='ROLLOVER',a.amount,0))-sum(coalesce(b.principal_paid,0)) as loan_balance,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(coalesce(b.principal_accr,0)) -(coalesce(b.principal_paid,0) ),0)) as total_current_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,

                (coalesce(b.principal_accr,0)) -(coalesce(b.principal_paid,0)),0)) as overdue_3_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_10_principle,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_30_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_60_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_90_principle,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,
                (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_more_than_90_principle,

                sum(coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + b.rollover_fee_paid + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)-b.erase_amount-b.coupon_amount) as total_paid_off_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD'),(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as total_outstanding_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as total_current_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_3_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_10_amount,

                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_30_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_60_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_90_amount,
                sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(now()+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
                (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_more_than_90_amount,'transfer' as mark

                from `banda-etl-s3`.`t_loan_app` a
                left  join `banda-etl-s3`.`t_lpay` b
                on a.`id` = b.`loan_app_id`
                where a.`status` in ('CURRENT','PAID_OFF','OVERDUE','GRACE_PERIOD') and a.product_name='RUPIAHONE' and a.`create_time` + interval '7' hour >= date '2019-08-26' 
                and a.customer_id not in (select customer_id from `banda-etl-s3`.t_test_customer)
                and a.id  in 
                (
                select rel.loan_id
                from `banda-etl-s3`.t_loan_tag_rel rel 
                where rel.tag_id in (select id from `banda-etl-s3`.t_tag where type='SELLING_CASE' ) and  rel.status='UNACTIVE'
                )




    """)
    df1.repartition(1).write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/market/loan_balance_summary1")


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1618893255944_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
spark.conf.set("spark.sql.adaptive.enabled", "false");
spark.sql("""
           with t_loan_app_not_in as(
        select a.* from `banda-etl-s3`.`t_loan_app` a 
            left join  
               (select rel.id, rel.status,rel.loan_id,tag.id as tagid from `banda-etl-s3`.t_loan_tag_rel rel 
            left join `banda-etl-s3`.t_tag tag
               on rel.tag_id=tag.id 
               and tag.type='SELLING_CASE'
               where tag.id is not null ) as tmp 
               on a.id =tmp.loan_id
               and  tmp.status='UNACTIVE'
            where tmp.loan_id is null 
            and  a.`status` in ('CURRENT','PAID_OFF','OVERDUE','GRACE_PERIOD') and a.product_name='RUPIAHONE' and a.`create_time` + interval '7' hour >= date '2019-08-26' 
            and a.customer_id not in (select customer_id from `banda-etl-s3`.t_test_customer)
            order by a.id desc 
                )

            select date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+interval '7' hour-interval '1' day) as date,
            sum(if(a.loan_type !='ROLLOVER',a.amount,0)) as total_issue_amount,

            sum(coalesce(b.principal_paid,0)) as total_paid_off_principle,

            sum(if(a.loan_type !='ROLLOVER',a.amount,0))-sum(coalesce(b.principal_paid,0)) as loan_balance,

            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(coalesce(b.principal_accr,0)) -(coalesce(b.principal_paid,0) ),0)) as total_current_principle,

            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,

            (coalesce(b.principal_accr,0)) -(coalesce(b.principal_paid,0)),0)) as overdue_3_principle,

            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,
            (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_10_principle,

            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,
            (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_30_principle,
            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,
            (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_60_principle,
            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,
            (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_90_principle,
            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,
            (coalesce(b.principal_accr,0) ) -(coalesce(b.principal_paid,0) ),0)) as overdue_more_than_90_principle,

            sum(coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + b.rollover_fee_paid + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)-b.erase_amount-b.coupon_amount) as total_paid_off_amount,

            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD'),(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
            (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as total_outstanding_amount,

            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour)) <=0,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
            (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as total_current_amount,
            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=1 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=3,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
            (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_3_amount,

            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=4 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=10,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
            (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_10_amount,

            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=11 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=30,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
            (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_30_amount,
            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=31 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=60,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
            (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_60_amount,
            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=61 and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))<=90,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
            (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_90_amount,
            sum(if(a.`status` in ('CURRENT','OVERDUE','GRACE_PERIOD') and datediff(date(date_format('2021-08-08 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+INTERVAL '7' hour),date(b.due_date+INTERVAL '7' hour))>=91,(coalesce(b.principal_accr,0) + coalesce(b.interest_accr,0) + coalesce(b.default_accr,0) + coalesce(b.default_fee_accr,0) + coalesce(b.service_fee_accr,0)) -
            (coalesce(b.principal_paid,0) + coalesce(b.interest_paid,0) + coalesce(b.default_paid,0) + coalesce(b.default_fee_paid,0) + coalesce(b.service_fee_paid,0)),0)) as overdue_more_than_90_amount,'transfer' as mark

            from `banda-etl-s3`.`t_loan_app` a
            left  join `banda-etl-s3`.`t_lpay` b
            on a.`id` = b.`loan_app_id`
            where a.`status` in ('CURRENT','PAID_OFF','OVERDUE','GRACE_PERIOD') and a.product_name='RUPIAHONE' and a.`create_time` + interval '7' hour >= date '2019-08-26' 
            and a.customer_id not in (select customer_id from `banda-etl-s3`.t_test_customer)
            and a.id  in 
            (
            select rel.loan_id
            from `banda-etl-s3`.t_loan_tag_rel rel 
            where rel.tag_id in (select id from `banda-etl-s3`.t_tag where type='SELLING_CASE' ) and  rel.status='ACTIVE'
            )
""").repartition(1).write.mode("append").orc("s3://rupiahplus-data-warehouse/etl/banda/market/loan_balance_summary2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark.sql("""   
   select date(date_format('2021-08-05 06:59:59','yyyy-MM-dd HH:mm:ss.SSS')+interval '7' hour-interval '1' day) as date
""").show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|      date|
+----------+
|2021-08-04|
+----------+